In [15]:
import numpy as np
import torch
from torch import tensor
from pathlib import Path
import pickle
from urllib.request import urlretrieve
import gzip

In [9]:
MNIST_URL='https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

In [10]:
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

In [11]:
with gzip.open(path_gz, "rb") as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [3]:
torch.manual_seed(1)

In [16]:
x_train, y_train, x_valid, y_valid = map(tensor, (x_train, y_train, x_valid, y_valid))

In [4]:
weights = torch.randn(784, 10)
bias = torch.zeros(10)

In [17]:
m1 = x_valid[:5]
m2 = weights

In [18]:
m1.shape, m2.shape

(torch.Size([5, 784]), torch.Size([784, 10]))

## Matrix multiplication - naive way

In [20]:
ar, ac = m1.shape
br, bc = m2.shape
(ar, ac), (br, bc)

((5, 784), (784, 10))

In [22]:
t1 = torch.zeros(ar, bc)
t1.shape

torch.Size([5, 10])

In [25]:
for i in range(ar):
    for j in range(bc):
        for k in range(ac):
            t1[i, j] += m1[i, k] * m2[k, j]

In [26]:
t1

tensor([[-10.9417,  -0.6844,  -7.0038,  -4.0066,  -2.0857,  -3.3588,   3.9127,
          -3.4375, -11.4696,  -2.1153],
        [ 14.5430,   5.9977,   2.8914,  -4.0777,   6.5914, -14.7383,  -9.2787,
           2.1577, -15.2772,  -2.6758],
        [  2.2204,  -3.2171,  -4.7988,  -6.0453,  14.1661,  -8.9824,  -4.7922,
          -5.4446, -20.6758,  13.5657],
        [ -6.7097,   8.8998,  -7.4611,  -7.8966,   2.6994,  -4.7260, -11.0278,
         -12.9776,  -6.4443,   3.6376],
        [ -2.4444,  -6.4034,  -2.3984,  -9.0371,  11.1772,  -5.7724,  -8.9214,
          -3.7862,  -8.9827,   5.2797]])

In [27]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
t1

tensor([[-10.94,  -0.68,  -7.00,  -4.01,  -2.09,  -3.36,   3.91,  -3.44, -11.47,  -2.12],
        [ 14.54,   6.00,   2.89,  -4.08,   6.59, -14.74,  -9.28,   2.16, -15.28,  -2.68],
        [  2.22,  -3.22,  -4.80,  -6.05,  14.17,  -8.98,  -4.79,  -5.44, -20.68,  13.57],
        [ -6.71,   8.90,  -7.46,  -7.90,   2.70,  -4.73, -11.03, -12.98,  -6.44,   3.64],
        [ -2.44,  -6.40,  -2.40,  -9.04,  11.18,  -5.77,  -8.92,  -3.79,  -8.98,   5.28]])

In [29]:
np.set_printoptions(precision=2, linewidth=140)

In [30]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(ac):
                c[i, j] += a[i, k] * b[k, j]
    return c

In [31]:
%time _ = matmul(m1, m2)

CPU times: user 292 ms, sys: 3.07 ms, total: 296 ms
Wall time: 296 ms


In [32]:
ar*bc*ac

39200

## Numba

In [33]:
from numba import njit

In [35]:
@njit
def dot(a,b):
    res = 0
    for i in range(len(a)):
        res += a[i] * b[i]
    return res

In [36]:
%time dot(np.array([1,2,3]), np.array([2,3,4]))

CPU times: user 1.62 s, sys: 346 ms, total: 1.97 s
Wall time: 720 ms


20

In [37]:
# it should be fast second time around as the code is now compiled
%time dot(np.array([1,2,3]), np.array([2,3,4]))

CPU times: user 50 µs, sys: 198 µs, total: 248 µs
Wall time: 260 µs


20

In [38]:
from fastcore.test import *

In [39]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            c[i,j] = dot(a[i, :], b[:, j])
    return c

In [40]:
m1a , m2a = m1.numpy(), m2.numpy()

In [41]:
test_close(t1, matmul(m1a, m2a))

In [43]:
%timeit -n 50 matmul(m1a, m2a)

272 µs ± 118 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


## Elementwise ops

In [70]:
a = tensor([10., 6, -4])
b = tensor([2., 8, 7])
a,b

(tensor([10.,  6., -4.]), tensor([2., 8., 7.]))

In [71]:
a + b

tensor([12., 14.,  3.])

In [72]:
(a < b).float().mean()

tensor(0.67)

In [73]:
m = tensor([[1,2,3],[4,5,6],[7,8,9]]); m

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

### Frobenius Norm
$$\| A \|_F = \left( \sum_{i,j=1}^n | a_{ij} |^2 \right)^{1/2}$$

In [51]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            c[i,j] = (a[i,:] * b[:,j]).sum()
    return c

In [58]:
test_close(t1, matmul(m1, m2))

In [59]:
%timeit -n 50  _ = matmul(m1, m2)

600 µs ± 249 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


In [60]:
def matmul(a,b):
    (ar,ac),(br,bc) = a.shape,b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc): c[i,j] = torch.dot(a[i,:], b[:,j])
    return c

In [61]:
test_close(t1,matmul(m1, m2))

In [62]:
%timeit -n 50 _=matmul(m1, m2)

380 µs ± 164 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


## Broadcasting

The term broadcasting describes how NumPy treats arrays with different shapes during arithmetic operations. 
- Subject to certain constraints, the smaller array is “broadcast” across the larger array so that they have compatible shapes.
- Broadcasting provides a means of vectorizing array operations so that looping occurs in C instead of Python.
- It does this without making needless copies of data and usually leads to efficient algorithm implementations.

### Rules:
When operating on two arrays, NumPy compares their shapes element-wise. It starts with the trailing (i.e. rightmost) dimension and works its way left. 
Two dimensions are compatible when

- they are equal, or
- one of them is 1.

Additionally,
- Input arrays do not need to have the same number of dimensions.
- The resulting array will have the same number of dimensions as the input array with the greatest number of dimensions, where the size of each dimension is the largest size of the corresponding dimension among the input arrays.
- Note that missing dimensions are assumed to have size one.


[Numpy documentation](https://numpy.org/doc/stable/user/basics.broadcasting.html)

In [69]:
## Outer product
x = np.array([0, 10, 20, 30])
y = np.array([1, 2, 3])
x[:, np.newaxis] + y

array([[ 1,  2,  3],
       [11, 12, 13],
       [21, 22, 23],
       [31, 32, 33]])

### Broadcasting with a scalar

In [74]:
a

tensor([10.,  6., -4.])

In [76]:
a > 0, a + 1, 2 * m

(tensor([ True,  True, False]),
 tensor([11.,  7., -3.]),
 tensor([[ 2,  4,  6],
         [ 8, 10, 12],
         [14, 16, 18]]))

### Broadcasting a vector to a matrix

In [77]:
c = tensor([10., 20, 30]); c

tensor([10., 20., 30.])

In [78]:
m

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [79]:
m.shape, c.shape

(torch.Size([3, 3]), torch.Size([3]))

In [80]:
m + c

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 28., 39.]])

In [81]:
t = c.expand_as(m); t

tensor([[10., 20., 30.],
        [10., 20., 30.],
        [10., 20., 30.]])

In [84]:
t.storage(), t.stride(), t.shape

/var/folders/33/q51v6n3n2rg4l3x1bqyznprh0000gn/T/ipykernel_69314/1846692255.py:1: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  t.storage(), t.stride(), t.shape


( 10.0
  20.0
  30.0
 [torch.storage.TypedStorage(dtype=torch.float32, device=cpu) of size 3],
 (0, 1),
 torch.Size([3, 3]))

In [88]:
c.unsqueeze(0).shape, c[None].shape

(torch.Size([1, 3]), torch.Size([1, 3]))

In [91]:
c.unsqueeze(1).shape, c[:, None].shape, c[...,None].shape

(torch.Size([3, 1]), torch.Size([3, 1]), torch.Size([3, 1]))

In [95]:
m + c[:, None], m + c[None, :]

(tensor([[11., 12., 13.],
         [24., 25., 26.],
         [37., 38., 39.]]),
 tensor([[11., 22., 33.],
         [14., 25., 36.],
         [17., 28., 39.]]))

In [96]:
## Outer product
c[None, :] * c[:, None]

tensor([[100., 200., 300.],
        [200., 400., 600.],
        [300., 600., 900.]])

## Matmul with broadcasting

In [99]:
m1[0, :, None].shape, m2.shape

(torch.Size([784, 1]), torch.Size([784, 10]))

In [102]:
(m1[0, :, None] *  m2).sum(dim=0)

tensor([-10.94,  -0.68,  -7.00,  -4.01,  -2.09,  -3.36,   3.91,  -3.44, -11.47,  -2.12])

In [104]:
m1.shape, m2.shape

(torch.Size([5, 784]), torch.Size([784, 10]))

In [105]:
def matmul(a, b):
    (ar, ac), (br, bc) = a.shape, b.shape
    c = torch.zeros(ar, bc)
    for i in range(ar):
        c[i] = (a[i, :, None] * b).sum(dim=0)
    return c

In [106]:
test_close(t1, matmul(m1, m2))

In [110]:
%timeit -n 50 _ = matmul(m1, m2)

126 µs ± 54 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


### 300 ms to 0.1 ms

In [112]:
# run on whole dataset
tr = matmul(x_train, weights)
tr.shape

torch.Size([50000, 10])

In [113]:
%time _ = matmul(x_train, weights)

CPU times: user 475 ms, sys: 3.05 ms, total: 478 ms
Wall time: 501 ms


## Einstein summation

In [114]:
mr = torch.einsum('ik, kj -> ikj', m1, m2)
mr.shape

torch.Size([5, 784, 10])

In [115]:
mr.sum(1)

tensor([[-10.94,  -0.68,  -7.00,  -4.01,  -2.09,  -3.36,   3.91,  -3.44, -11.47,  -2.12],
        [ 14.54,   6.00,   2.89,  -4.08,   6.59, -14.74,  -9.28,   2.16, -15.28,  -2.68],
        [  2.22,  -3.22,  -4.80,  -6.05,  14.17,  -8.98,  -4.79,  -5.44, -20.68,  13.57],
        [ -6.71,   8.90,  -7.46,  -7.90,   2.70,  -4.73, -11.03, -12.98,  -6.44,   3.64],
        [ -2.44,  -6.40,  -2.40,  -9.04,  11.18,  -5.77,  -8.92,  -3.79,  -8.98,   5.28]])

In [116]:
torch.einsum('ik, kj -> ij', m1, m2)

tensor([[-10.94,  -0.68,  -7.00,  -4.01,  -2.09,  -3.36,   3.91,  -3.44, -11.47,  -2.12],
        [ 14.54,   6.00,   2.89,  -4.08,   6.59, -14.74,  -9.28,   2.16, -15.28,  -2.68],
        [  2.22,  -3.22,  -4.80,  -6.05,  14.17,  -8.98,  -4.79,  -5.44, -20.68,  13.57],
        [ -6.71,   8.90,  -7.46,  -7.90,   2.70,  -4.73, -11.03, -12.98,  -6.44,   3.64],
        [ -2.44,  -6.40,  -2.40,  -9.04,  11.18,  -5.77,  -8.92,  -3.79,  -8.98,   5.28]])

In [117]:
def matmul(a, b):
    return torch.einsum('ik, kj->ij', a, b)

In [118]:
test_close(tr, matmul(x_train, weights), eps=1e-3)

In [119]:
%timeit -n 5 _=matmul(x_train, weights)

11 ms ± 6.21 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


## Pytorch op

In [121]:
test_close(tr, x_train @ weights, eps=1e-3)

In [122]:
%timeit -n 5 _=torch.matmul(x_train, weights)

10.9 ms ± 5.91 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


## CUDA

In [123]:
def matmul(grid, a,b,c):
    i,j = grid
    if i < c.shape[0] and j < c.shape[1]:
        tmp = 0.
        for k in range(a.shape[1]): tmp += a[i, k] * b[k, j]
        c[i,j] = tmp
     

In [124]:
res = torch.zeros(ar, bc)
matmul((0,0), m1, m2, res)
res

tensor([[-10.94,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00],
        [  0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00],
        [  0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00],
        [  0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00],
        [  0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00,   0.00]])

In [125]:
def launch_kernel(kernel, grid_x, grid_y, *args, **kwargs):
    for i in range(grid_x):
        for j in range(grid_y): 
            kernel((i,j), *args, **kwargs)

In [126]:
res = torch.zeros(ar, bc)
launch_kernel(matmul, ar, bc, m1, m2, res)
res

tensor([[-10.94,  -0.68,  -7.00,  -4.01,  -2.09,  -3.36,   3.91,  -3.44, -11.47,  -2.12],
        [ 14.54,   6.00,   2.89,  -4.08,   6.59, -14.74,  -9.28,   2.16, -15.28,  -2.68],
        [  2.22,  -3.22,  -4.80,  -6.05,  14.17,  -8.98,  -4.79,  -5.44, -20.68,  13.57],
        [ -6.71,   8.90,  -7.46,  -7.90,   2.70,  -4.73, -11.03, -12.98,  -6.44,   3.64],
        [ -2.44,  -6.40,  -2.40,  -9.04,  11.18,  -5.77,  -8.92,  -3.79,  -8.98,   5.28]])

In [130]:
@cuda.jit
def matmul(a,b,c):
    i, j = cuda.grid(2)
    if i < c.shape[0] and j < c.shape[1]:
        tmp = 0.
        for k in range(a.shape[1]): tmp += a[i, k] * b[k, j]
        c[i,j] = tmp


In [ ]:
r = np.zeros(tr.shape)
m1g,m2g,rg = map(cuda.to_device, (x_train,weights,r))

In [ ]:
TPB = 16
rr,rc = r.shape
blockspergrid = (math.ceil(rr / TPB), math.ceil(rc / TPB))
blockspergrid

In [ ]:
matmul[blockspergrid, (TPB,TPB)](m1g,m2g,rg)
r = rg.copy_to_host()
test_close(tr, r, eps=1e-3)

In [ ]:
%%timeit -n 10
matmul[blockspergrid, (TPB,TPB)](m1g,m2g,rg)
r = rg.copy_to_host()

In [ ]:
m1c,m2c = x_train.cuda(),weights.cuda()

In [ ]:
%timeit -n 10 r=(m1c@m2c).cpu()